In [48]:
import pandas as pd
import numpy as np

In [5]:
sales = pd.read_csv("../clean_data/weekly_sales_total.csv")

In [6]:
sales

,Date,Weekly_Sales,Week
0,2010-02-05,11694898.08,5
1,2010-02-12,11338849.87,6
2,2010-02-19,11495206.55,7
3,2010-02-26,10401334.38,8
4,2010-03-05,11030603.21,9
...,...,...,...
138,2012-09-28,9801584.18,39
139,2012-10-05,10436585.09,40
140,2012-10-12,10208583.45,41
141,2012-10-19,10057643.14,42


In [20]:
sales_2010 = sales.query("Date < '2011-01-01'").copy(deep=True).rename(columns={'Date': '2010', 'Weekly_Sales': 'Sales_2010'})
sales_2011 = sales.query("'2011-01-01' <= Date < '2012-01-01'").copy(deep=True).rename(columns={'Date': '2011', 'Weekly_Sales': 'Sales_2011'})
sales_2012 = sales.query("'2012-01-01' <= Date < '2013-01-01'").copy(deep=True).rename(columns={'Date': '2012', 'Weekly_Sales': 'Sales_2012'})

In [37]:
sales_by_year = pd.merge(sales_2010, sales_2011, how='outer')
sales_by_year = pd.merge(sales_by_year, sales_2012, how='outer')
sales_by_year = sales_by_year.loc[:,['Week', '2010', '2011', '2012', 'Sales_2010', 'Sales_2011', 'Sales_2012']]
sales_by_year = sales_by_year.sort_values(by='Week')
sales_by_year.tail(16)

,Week,2010,2011,2012,Sales_2010,Sales_2011,Sales_2012
32,37,2010-09-17,2011-09-16,2012-09-14,9553476.54,9354382.06,9544108.66
33,38,2010-09-24,2011-09-23,2012-09-21,9176907.35,9374616.73,9720423.99
34,39,2010-10-01,2011-09-30,2012-09-28,9341212.98,9340027.21,9801584.18
35,40,2010-10-08,2011-10-07,2012-10-05,10064797.88,10229117.79,10436585.09
36,41,2010-10-15,2011-10-14,2012-10-12,9525027.36,9666920.39,10208583.45
37,42,2010-10-22,2011-10-21,2012-10-19,9756376.71,10318466.03,10057643.14
38,43,2010-10-29,2011-10-28,2012-10-26,9977792.18,10568047.84,10386208.11
39,44,2010-11-05,2011-11-04,NaN,10216271.81,10777647.16,NaN
40,45,2010-11-12,2011-11-11,NaN,10577447.84,11119605.10,NaN
41,46,2010-11-19,2011-11-18,NaN,10363491.89,10450311.11,NaN


El crecimiento parece bastante similar entre el año 2010 y 2011, y la variación entre 2011 y 2012, que sucede si realizamos la predicción basado en este crecimiento?

In [39]:
sales_by_year['grow_10_11'] = sales_by_year['Sales_2011'] / sales_by_year['Sales_2010']
sales_by_year['fcst_2012'] = sales_by_year['Sales_2011'] * sales_by_year['grow_10_11']

### Calculo de error en método super simple
Utilizare [sMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) para calcular el error del forecast de 7 semanas.
Es basado en porcentaje, independiente de la escala, y se puede utilizar para comprar performance de forecast.


In [52]:
fcst_prev_7_weeks = sales_by_year.query("37 <= Week <= 43")
fcst_prev_7_weeks = fcst_prev_7_weeks.loc[:,['Week', 'Sales_2012', 'fcst_2012']]
fcst_prev_7_weeks['smape'] = abs(fcst_prev_7_weeks['fcst_2012'] - fcst_prev_7_weeks['Sales_2012']) / ((abs(fcst_prev_7_weeks['fcst_2012']) + abs(fcst_prev_7_weeks['Sales_2012']))/2)
fcst_prev_7_weeks

,Week,Sales_2012,fcst_2012,smape
32,37,9544108.66,9.159437e+06,0.041134
33,38,9720423.99,9.576586e+06,0.014908
34,39,9801584.18,9.338842e+06,0.048352
35,40,10436585.09,1.039612e+07,0.003885
36,41,10208583.45,9.810927e+06,0.039727
37,42,10057643.14,1.091294e+07,0.081571
38,43,10386208.11,1.119322e+07,0.074795


In [57]:
round(np.mean(fcst_prev_7_weeks['smape'].values)*100,2)

4.35

Aplicar este método para las 7 semanas anterior a las solicitadas, tendría un error sMAPE de 4.35%, o una precisión del 95.65%

Para este caso el forecast de las 7 semanas solicitadas (44 a la 50) sería:

In [63]:
fcst_next_7_weeks = sales_by_year.query("44 <= Week <= 50").copy(deep=True)
fcst_next_7_weeks['fcst_2012'] = fcst_next_7_weeks['fcst_2012'].astype('int')
fcst_next_7_weeks = fcst_next_7_weeks.loc[:,['Week', 'fcst_2012']]
fcst_next_7_weeks

,Week,fcst_2012
39,44,11369869
40,45,11689551
41,46,10537857
42,47,16716822
43,48,11352270
44,49,12844111
45,50,13870741
